In [2]:
# ===========================================================
# End-to-end: Predict -> Export -> TrackEval -> Official metrics
# ===========================================================

# ---- numpy compatibility shim (put at top, BEFORE importing trackeval) ----
import numpy as np

# Add removed numpy aliases back (safe: maps to built-in types)
# Only add the ones you need to avoid masking real problems
if not hasattr(np, "float"):
    np.float = float
if not hasattr(np, "int"):
    np.int = int
if not hasattr(np, "bool"):
    np.bool = bool
# ---------------------------------------------------------------------------

import os
from pathlib import Path
import json
# import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from collections import defaultdict
from textwrap import dedent # ADDED for minimal yaml generation

# --- CONFIG (edit paths as needed) ---
IMAGE_DIR   = Path(r"C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K\videos\val")
GT_ROOT     = Path(r"C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K\labelsMOT")  # contains label_02/
LABEL_DIR   = GT_ROOT / "val"
OUTPUT_DIR  = Path(r"C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking") # CHANGED OUTPUT_DIR for cleaner separation

MODEL_PATH  = Path('BDD1280_ep6.pt')
TRACKER_YAML= "botsort.yaml"  # CHANGED to botsort.yaml
CONF_THRESH = 0.1
SAVE_VIDEOS = True
TRACKER_NAME= "SpikeYOLO"

# Optional: limit sequences for a quick pass (None = all)
MAX_SEQUENCES = None    # e.g., 3

# --- Create folders ---
(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / "videos").mkdir(exist_ok=True)
(OUTPUT_DIR / "predictions").mkdir(exist_ok=True)
(OUTPUT_DIR / "metrics").mkdir(exist_ok=True)

# TrackEval IO layout
TE_ROOT           = OUTPUT_DIR / "trackeval_io"
TE_TRACKERS_ROOT = TE_ROOT / "trackers" / "KITTI_2D_BOX"
TE_TRKDATA       = TE_TRACKERS_ROOT / TRACKER_NAME / "data"
TE_TRKDATA.mkdir(parents=True, exist_ok=True)

RAW_OUT = OUTPUT_DIR / "metrics" / "trackeval_raw"
RAW_OUT.mkdir(parents=True, exist_ok=True)

SEQMAP_FILE = GT_ROOT / "evaluate_tracking.seqmap.training"

# --- Imports that can be heavy ---
from ultralytics import YOLO
import trackeval
from scipy.optimize import linear_sum_assignment  # for any fallback matching (not strictly needed now)

# ========== 1) Utilities ==========

CLASS_MAPPING = {
    'Car': 'Car', 'Van': 'Car', 'Truck': 'Car', 'Tram': 'Car',
    'Pedestrian': 'Pedestrian', 'Person_sitting': 'Pedestrian',
    'Cyclist': 'IGNORE', 'Misc': 'IGNORE', 'DontCare': 'IGNORE'
}
VALID_CLASSES = ['Car', 'Pedestrian']

def get_sequences(image_dir: Path):
    seqs = sorted([d.name for d in image_dir.iterdir() if d.is_dir()])
    if MAX_SEQUENCES is not None:
        seqs = seqs[:MAX_SEQUENCES]
    print(f"Found {len(seqs)} sequences: {seqs[:5]}{' ...' if len(seqs)>5 else ''}")
    return seqs

def setup_model_mapping(model):
    """Map model classes to KITTI classes."""
    model_to_kitti = {}
    target_ids       = []
    print("Detected model classes:", model.names)
    for cls_id, name in model.names.items():
        n = name.lower().strip()
        if n in ['car','van','truck','tram','vehicle']:
            model_to_kitti[cls_id] = 'Car'
            target_ids.append(cls_id)
        elif n in ['pedestrian','person','person_sitting']:
            model_to_kitti[cls_id] = 'Pedestrian'
            target_ids.append(cls_id)
    print("Class mapping (model -> KITTI):", {model.names[i]:model_to_kitti[i] for i in model_to_kitti})
    return model_to_kitti, target_ids

def load_kitti_labels(sequence_id: str):
    file = LABEL_DIR / f"{sequence_id}.txt"
    if not file.exists():  
        return None
    rows = []
    with open(file, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 10:  
                continue
            frame = int(p[0]); tid = int(p[1]); obj = p[2]
            if obj not in CLASS_MAPPING:  
                continue
            mapped = CLASS_MAPPING[obj]
            if mapped == "IGNORE":  
                continue
            x1,y1,x2,y2 = map(float, p[6:10])
            rows.append({
                "frame": frame, "id": tid, "type": mapped,
                "x": x1, "y": y1, "w": x2-x1, "h": y2-y1
            })
    df = pd.DataFrame(rows)
    return df

def export_seq_to_kitti_result(pred_df: pd.DataFrame, sequence_id: str):
    """
    KITTI 2D tracking results format (per-seq file):
    frame id type truncated occluded alpha x1 y1 x2 y2 h w l X Y Z ry score
    Unknown 3D fields filled with sentinels.
    """
    out = TE_TRKDATA / f"{sequence_id}.txt"
    rows = []
    for _, r in pred_df.iterrows():
        x1 = float(r['x']); y1 = float(r['y'])
        x2 = float(r['x'] + r['w']); y2 = float(r['y'] + r['h'])
        rows.append([
            int(r['frame']), int(r['id']), str(r['type']),
            -1, -1, -10, x1, y1, x2, y2,
            -1, -1, -1, -1000, -1000, -1000, -10,
            float(r.get('conf', 1.0))
        ])
    rows.sort(key=lambda R: (R[0], R[1]))
    with out.open("w") as f:
        for R in rows:
            f.write(
                f"{R[0]} {R[1]} {R[2]} {R[3]} {R[4]} {R[5]} "
                f"{R[6]:.2f} {R[7]:.2f} {R[8]:.2f} {R[9]:.2f} "
                f"{R[10]} {R[11]} {R[12]} {R[13]} {R[14]} {R[15]} {R[16]} {R[17]:.6f}\n"
            )
    return out

def build_seqmap_for_present_sequences():
    """
    Build seqmap only for sequences that have BOTH GT and tracker files.
    """
    trk_seqs = sorted(p.stem for p in TE_TRKDATA.glob("*.txt"))
    gt_seqs  = sorted(p.stem for p in LABEL_DIR.glob("*.txt"))
    present  = sorted(set(trk_seqs).intersection(gt_seqs))
    if not present:
        raise RuntimeError("No overlapping sequences between tracker results and GT.")
    rows = []
    for seq in present:
        gt_file = LABEL_DIR / f"{seq}.txt"
        df = pd.read_csv(gt_file, sep=r"\s+", header=None, usecols=[0], engine="python")
        if df.empty:  
            continue
        max_f = int(df.iloc[:,0].max())
        rows.append((seq, 0, max_f+1))
    if not rows:
        raise RuntimeError("No non-empty GT sequences to evaluate.")
    with SEQMAP_FILE.open("w") as f:
        for seq, s, e in rows:
            f.write(f"{seq} empty {s:06d} {e:06d}\n")
    print(f"✓ Wrote seqmap with {len(rows)} sequences at: {SEQMAP_FILE}")

def _color_for_class(class_name: str, track_id: int):
    np.random.seed(track_id)
    base = np.array([0, 255, 0]) if class_name == 'Car' else np.array([255, 100, 0])
    return tuple(map(int, np.clip(base + np.random.randint(-30, 30, 3), 0, 255)))

# ADDED utility to ensure correct video format
def _ensure_bgr_uint8(img):
    if img is None:
        return None
    if img.dtype != np.uint8:
        img = np.clip(img, 0, 255).astype(np.uint8)
    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    elif img.ndim == 3 and img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return img

# ========== 2) Predict all sequences (YOLO+BOTSort; stream + GMC disabled; skip processed) ==========
print("\n=== Running predictions (YOLO + BOTSort) ===")
sequences = get_sequences(IMAGE_DIR)
model = YOLO(str(MODEL_PATH))
model_to_kitti, target_ids = setup_model_mapping(model)

# --- Prepare a temporary tracker YAML with GMC turned off ---
tmp_tracker_yaml = OUTPUT_DIR / "botsort_no_gmc.yaml"
trk_path = Path(TRACKER_YAML)

# These parameters are required by the BoT-SORT class constructor, even if not used (due to `with_reid: false`)
BOT_SORT_DEFAULTS = """
proximity_thresh: 0.5
appearance_thresh: 0.25
"""

# The parameter the tracker class looks for is the flat key 'gmc_method'
GMC_OVERRIDE = "gmc_method: none"


if trk_path.exists():
    # Read original and modify it
    with open(trk_path, "r") as f:
        orig_yaml = f.read()

    # 1. Ensure required ReID defaults are present (fixes prior error)
    new_yaml = orig_yaml
    for line in BOT_SORT_DEFAULTS.strip().split('\n'):
        key = line.split(':')[0].strip()
        if f"{key}:" not in new_yaml:
            new_yaml += f"\n{line}"

    # 2. Add or overwrite the flat 'gmc_method' parameter for KITTI compatibility
    lines = new_yaml.splitlines()
    found_gmc_method = False
    final_lines = []
    
    for ln in lines:
        if ln.strip().startswith("gmc_method:"):
            # Overwrite existing flat gmc_method
            final_lines.append(GMC_OVERRIDE)
            found_gmc_method = True
        elif ln.strip().startswith("gmc:"):
            # Skip the nested 'gmc:' structure if it exists, as we use the flat key
            continue
        else:
            final_lines.append(ln)

    new_yaml = "\n".join(final_lines)

    # If gmc_method was never found, append it
    if not found_gmc_method:
        new_yaml += f"\n{GMC_OVERRIDE}\n"

else:
    # No tracker YAML provided -> generate a minimal, sane BoT-SORT config with GMC off
    new_yaml = dedent(f"""
        tracker_type: botsort
        track_high_thresh: 0.1
        track_low_thresh: 0.05
        new_track_thresh: 0.2
        match_thresh: 0.8
        track_buffer: 30
        with_reid: false
        # REQUIRED to fix the previous AttributeError
        proximity_thresh: 0.5
        appearance_thresh: 0.25
        # REQUIRED to fix the current AttributeError and disable GMC for KITTI
        {GMC_OVERRIDE}
    """).strip()

with open(tmp_tracker_yaml, "w") as f:
    f.write(new_yaml)

print(f"Using tracker config: {tmp_tracker_yaml}")

all_pred_files = []
for seq_id in sequences:
    print(f"\n--- Sequence {seq_id} ---")
    seq_dir = IMAGE_DIR / seq_id
    imgs = sorted(seq_dir.glob("*.jpg"))
    if not imgs:
        print("  (no images) -> skip")
        continue

    # --- Skip if already processed ---
    pred_csv_path = OUTPUT_DIR / "predictions" / f"{seq_id}_predictions.csv"
    te_file_path  = TE_TRKDATA / f"{seq_id}.txt"
    video_path    = OUTPUT_DIR / "videos" / f"{seq_id}_tracked.mp4"
    already_done  = pred_csv_path.exists() and te_file_path.exists() and (not SAVE_VIDEOS or video_path.exists())
    if already_done:
        print("  ✓ Already processed (found predictions + TrackEval file"
              f"{' + video' if SAVE_VIDEOS else ''}). Skipping.")
        # keep list for downstream checks (so Step 3/4 can still run)
        all_pred_files.append(pred_csv_path)
        continue

    # Reset any lingering tracker state (IMPORTANT for sequence-by-sequence tracking)
    try:
        if hasattr(model, "predictor") and hasattr(model.predictor, "reset_tracking"):
            model.predictor.reset_tracking()
    except Exception:
        pass

    # Probe first frame to set video size
    probe = _ensure_bgr_uint8(cv2.imread(str(imgs[0]), cv2.IMREAD_COLOR))
    if probe is None:
        print(f"  (cannot read first frame: {imgs[0]}) -> skip")
        continue
    ref_h, ref_w = probe.shape[:2]

    # Video writer
    vw = None
    if SAVE_VIDEOS:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        vw = cv2.VideoWriter(str(video_path), fourcc, 10, (ref_w, ref_h))

    preds = []
    frame_idx = -1

    # Single efficient streaming call over the folder (REPLACED per-frame loop)
    for res in tqdm(
        model.track(
            source=str(seq_dir),
            conf=CONF_THRESH,
            iou=0.5,
            tracker=str(tmp_tracker_yaml),          # GMC disabled
            classes=target_ids if target_ids else None,
            persist=False,
            verbose=False,
            stream=True
        ),
        total=len(imgs),
        desc=f"Tracking {seq_id}"
    ):
        frame_idx += 1
        frame = _ensure_bgr_uint8(res.orig_img)
        if frame.shape[:2] != (ref_h, ref_w):
            frame = cv2.resize(frame, (ref_w, ref_h), interpolation=cv2.INTER_LINEAR)

        boxes = res.boxes
        if boxes is not None and boxes.id is not None:
            xyxy = boxes.xyxy.cpu().numpy()
            tids = boxes.id.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            clss = boxes.cls.cpu().numpy()
            for box, tid, conf, cls_idx in zip(xyxy, tids, confs, clss):
                x1, y1, x2, y2 = box
                k = int(cls_idx)
                if k not in model_to_kitti:
                    continue
                kitti_cls = model_to_kitti[k]
                preds.append({
                    "frame": frame_idx,
                    "id": int(tid),
                    "type": kitti_cls,
                    "x": float(x1),
                    "y": float(y1),
                    "w": float(x2 - x1),
                    "h": float(y2 - y1),
                    "conf": float(conf)
                })
                if SAVE_VIDEOS:
                    color = _color_for_class(kitti_cls, int(tid))
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                    label = f"{kitti_cls} ID:{int(tid)} {conf:.2f}"
                    cv2.putText(frame, label, (int(x1), int(y1) - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        if SAVE_VIDEOS and vw is not None:
            vw.write(frame)

    if SAVE_VIDEOS and vw is not None:
        vw.release()
        print(f"  ✓ Video saved: {video_path}")

    pred_df = pd.DataFrame(preds)
    if pred_df.empty:
        print("  (no predictions) -> skip export")
        continue

    # Save predictions CSV
    pred_df.to_csv(pred_csv_path, index=False)
    all_pred_files.append(pred_csv_path)
    print(f"  ✓ Predictions CSV: {pred_csv_path} ({len(pred_df)} dets)")

    # Export to TrackEval KITTI result file
    te_file = export_seq_to_kitti_result(pred_df, seq_id)
    print(f"  ✓ TrackEval file: {te_file}")

if not all_pred_files:
    raise RuntimeError("No predictions produced; check model path / tracker / confidence threshold.")


# ========== 3) Build seqmap (reconcile GT & tracker files) ==========
build_seqmap_for_present_sequences()

# ========== 4) Run TrackEval (official) ==========
print("\n=== Running TrackEval (official KITTI_2D_BOX) ===")
dataset_config = {
    'GT_FOLDER'         : str(GT_ROOT),
    'TRACKERS_FOLDER' : str(TE_TRACKERS_ROOT),  # includes 'KITTI_2D_BOX'
    'OUTPUT_FOLDER'   : str(RAW_OUT),
    'TRACKERS_TO_EVAL': [TRACKER_NAME],
    'BENCHMARK'       : 'KITTI_2D_BOX',
    'SEQMAP_FILE'     : str(SEQMAP_FILE),
    'DO_PREPROC'      : False,
    'CLASSES_TO_EVAL' : ['Car', 'Pedestrian'],
}
eval_config = {
    'USE_PARALLEL': False,
    'NUM_PARALLEL_CORES': 1,
    'PRINT_RESULTS': True,
    'PRINT_ONLY_COMBINED': False,
    'TIME_PROGRESS': False,
    'DISPLAY_LESS_PROGRESS': True,
}
metrics_list = [trackeval.metrics.HOTA(), trackeval.metrics.CLEAR(), trackeval.metrics.Identity()]
evaluator    = trackeval.Evaluator(eval_config)
dataset_list = [trackeval.datasets.Kitti2DBox(dataset_config)]
output_res, _ = evaluator.evaluate(dataset_list, metrics_list)

# ========== 5) Parse nested results and compute official AUCs ==========
# Your TrackEval build nests metrics by class:
# output_res['Kitti2DBox']['SpikeYOLO'][seq]['car' or 'pedestrian'] has:
#    'HOTA': {'HOTA': np.ndarray, 'DetA': np.ndarray, 'AssA': np.ndarray, 'LocA': np.ndarray, ...}
#    'CLEAR': {'MOTA': float, 'MOTP': float, ...}
#    'Identity': {'IDF1': float, 'IDP': float, 'IDR': float}
#
# Official "AUC" for HOTA/DetA/AssA is the MEAN of those arrays across thresholds.

def arr_mean(x):
    try:
        x = np.asarray(x)
        if x.ndim == 0:
            return float(x)
        return float(np.nanmean(x))
    except Exception:
        return None

CLASS_NAME_MAP = {"car": "Car", "pedestrian": "Pedestrian"}

rows = []
for dataset_name, trackers in output_res.items():
    for tracker_name, seqs in trackers.items():
        for seq_name, metrics in seqs.items():
            for cls_key, cls_block in metrics.items():
                if not isinstance(cls_block, dict):
                    continue
                cls_norm = str(cls_key).lower()
                if cls_norm not in CLASS_NAME_MAP:
                    continue
                pretty_cls = CLASS_NAME_MAP[cls_norm]

                # HOTA group
                HOTA_grp = cls_block.get("HOTA", {})
                HOTA_auc = arr_mean(HOTA_grp.get("HOTA"))
                DetA_auc = arr_mean(HOTA_grp.get("DetA"))
                AssA_auc = arr_mean(HOTA_grp.get("AssA"))
                LocA_auc = arr_mean(HOTA_grp.get("LocA")) if "LocA" in HOTA_grp else None

                # CLEAR group
                CLEAR = cls_block.get("CLEAR", {})
                MOTA  = CLEAR.get("MOTA", None)
                MOTP  = CLEAR.get("MOTP", None)
                MOTA  = float(MOTA) if MOTA is not None else None
                MOTP  = float(MOTP) if MOTP is not None else None

                # Identity group
                ID = cls_block.get("Identity", {})
                IDF1 = float(ID.get("IDF1")) if "IDF1" in ID else None
                IDP  = float(ID.get("IDP"))  if "IDP"  in ID else None
                IDR  = float(ID.get("IDR"))  if "IDR"  in ID else None

                rows.append({
                    "Sequence": seq_name,
                    "Class": pretty_cls,
                    "HOTA": HOTA_auc,
                    "DetA": DetA_auc,
                    "AssA": AssA_auc,
                    "LocA": LocA_auc,
                    "MOTA": MOTA,
                    "MOTP": MOTP,
                    "IDF1": IDF1,
                    "IDP": IDP,
                    "IDR": IDR
                })

metrics_df = pd.DataFrame(rows).sort_values(["Sequence", "Class"])
if metrics_df.empty:
    raise RuntimeError("Parsed metrics are empty. The expected nested keys changed; print output_res to inspect.")

# Save per-seq, per-class metrics
per_seq_csv = OUTPUT_DIR / "metrics" / "trackeval_per_sequence_per_class.csv"
metrics_df.to_csv(per_seq_csv, index=False)
print(f"\n✓ Saved per-seq, per-class metrics: {per_seq_csv}")

# ========== 6) Print per-sequence with BOTH classes side-by-side ==========
def print_seq(seq):
    d = metrics_df[metrics_df["Sequence"]==seq]
    if d.empty:  
        return
    cols = ["Class", "HOTA","DetA","AssA","MOTA","MOTP","IDF1"]
    present = ["Class"] + [c for c in cols if c in d.columns and c != "Class"]
    tab = d[present].set_index("Class")
    # Show decimals (0-1 range) like TrackEval default
    print(f"\nSequence {seq} — OFFICIAL metrics (AUC where applicable)")
    display(tab)

print("\n=== Per-sequence OFFICIAL metrics ===")
for seq in sorted(metrics_df["Sequence"].unique()):
    print_seq(seq)

# ========== 7) Class-wise averages (mean ± std across sequences) ==========
agg = (metrics_df
       .groupby("Class")
       .agg({c:["mean","std"] for c in ["HOTA","DetA","AssA","MOTA","MOTP","IDF1","IDP","IDR"] if c in metrics_df.columns})
       .sort_index())
agg.columns = [f"{m}_{stat}" for m,stat in agg.columns]
class_avg_csv = OUTPUT_DIR / "metrics" / "trackeval_class_averages.csv"
agg.to_csv(class_avg_csv)
print("\n=== Class-wise averages (OFFICIAL) ===")
display(agg)
print(f"✓ Saved class averages: {class_avg_csv}")

# Also save compact JSON
class_avg_json = OUTPUT_DIR / "metrics" / "trackeval_class_averages.json"
class_json = {}
for cls, row in agg.iterrows():
    stats = {}
    for m in ["HOTA","DetA","AssA","MOTA","MOTP","IDF1","IDP","IDR"]:
        mean_col = f"{m}_mean"; std_col = f"{m}_std"
        if mean_col in agg.columns:
            stats[m] = {"mean": float(row[mean_col]), "std": float(row.get(std_col, np.nan))}
    class_json[cls] = stats
with open(class_avg_json, "w") as f:
    json.dump(class_json, f, indent=2)
print(f"✓ Saved class averages JSON: {class_avg_json}")

# ========== 8) Dataset-wide combined (macro) ==========
# Average across all (seq, class) rows
combined = metrics_df.drop(columns=["Sequence","Class"]).mean(numeric_only=True)
print("\n=== Combined (macro average over sequences & classes) ===")
print(combined.to_string(float_format=lambda x: f"{x:0.4f}"))

combined_csv = OUTPUT_DIR / "metrics" / "trackeval_combined_macro.csv"
combined.to_csv(combined_csv)
print(f"✓ Saved combined macro averages: {combined_csv}")


=== Running predictions (YOLO + BOTSort) ===
Found 50 sequences: ['b1c81faa-3df17267', 'b1cd1e94-549d0bfe', 'b1ceb32e-3f481b43', 'b1ceb32e-51852abe', 'b1d0091f-f2c2d2ae'] ...
Detected model classes: {0: 'car', 1: 'pedestrian'}
Class mapping (model -> KITTI): {'car': 'Car', 'pedestrian': 'Pedestrian'}
Using tracker config: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\botsort_no_gmc.yaml

--- Sequence b1c81faa-3df17267 ---


Tracking b1c81faa-3df17267:   0%|                                                              | 0/202 [00:00<?, ?it/s]C:\Users\Manish\miniconda3\Lib\site-packages\torch\functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4319.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Tracking b1c81faa-3df17267: 100%|████████████████████████████████████████████████████| 202/202 [00:35<00:00,  5.61it/s]


  ✓ Video saved: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\videos\b1c81faa-3df17267_tracked.mp4
  ✓ Predictions CSV: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\predictions\b1c81faa-3df17267_predictions.csv (391 dets)
  ✓ TrackEval file: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\trackeval_io\trackers\KITTI_2D_BOX\SpikeYOLO\data\b1c81faa-3df17267.txt

--- Sequence b1cd1e94-549d0bfe ---


Tracking b1cd1e94-549d0bfe: 100%|████████████████████████████████████████████████████| 203/203 [00:36<00:00,  5.52it/s]


  ✓ Video saved: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\videos\b1cd1e94-549d0bfe_tracked.mp4
  ✓ Predictions CSV: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\predictions\b1cd1e94-549d0bfe_predictions.csv (1201 dets)
  ✓ TrackEval file: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\trackeval_io\trackers\KITTI_2D_BOX\SpikeYOLO\data\b1cd1e94-549d0bfe.txt

--- Sequence b1ceb32e-3f481b43 ---


...

Tracking b251064f-30002542: 100%|████████████████████████████████████████████████████| 202/202 [00:34<00:00,  5.82it/s]


  ✓ Video saved: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\videos\b251064f-30002542_tracked.mp4
  ✓ Predictions CSV: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\predictions\b251064f-30002542_predictions.csv (1531 dets)
  ✓ TrackEval file: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\trackeval_io\trackers\KITTI_2D_BOX\SpikeYOLO\data\b251064f-30002542.txt

--- Sequence b255cd6c-0bdf0ac7 ---


Tracking b255cd6c-0bdf0ac7: 100%|████████████████████████████████████████████████████| 202/202 [00:34<00:00,  5.84it/s]


  ✓ Video saved: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\videos\b255cd6c-0bdf0ac7_tracked.mp4
  ✓ Predictions CSV: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\predictions\b255cd6c-0bdf0ac7_predictions.csv (598 dets)
  ✓ TrackEval file: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\trackeval_io\trackers\KITTI_2D_BOX\SpikeYOLO\data\b255cd6c-0bdf0ac7.txt


RuntimeError: No overlapping sequences between tracker results and GT.

In [8]:
import os
import pandas as pd
import json
from pathlib import Path
from tqdm import tqdm

# ============================================================================
# CONFIGURATION
# ============================================================================

# Base directory setup (matches other scripts)
base_dir = Path(r"C:\Users\Manish\Documents\SpikeYOLOALL")

# --- INPUT CONFIG ---
# Directory containing your BDD100K prediction JSONs (one file per sequence).
# Example filename: '100000.json'
INPUT_JSON_DIR = Path(r"C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K\labelsMOT\val")

# --- OUTPUT CONFIG ---
# The target directory for TrackEval's KITTI 2D Box loader (TE_TRKDATA)
TRACKER_NAME = "BDD_JSON_TO_KITTI_CONV" # Set a unique name for this converted tracker
TE_ROOT = base_dir / "BDD100K_tracking" / "trackeval_io"
TE_TRACKERS_ROOT = TE_ROOT / "trackers" / "KITTI_2D_BOX"
OUTPUT_TRKDATA_DIR = TE_TRACKERS_ROOT / TRACKER_NAME / "data"

# Map BDD categories to the required KITTI categories
BDD_CLASS_MAP = {
    'car': 'Car', 'bus': 'Car', 'truck': 'Car', 
    'pedestrian': 'Pedestrian', 'person': 'Pedestrian',
    'rider': 'IGNORE', 'motorcycle': 'IGNORE', # Ignore other classes
}

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def load_and_convert_sequence(json_path: Path) -> pd.DataFrame:
    """
    Reads a BDD100K format JSON (list of frame objects) and converts it 
    into a flat DataFrame required for KITTI export.
    """
    with json_path.open('r') as f:
        try:
            bdd_frames = json.load(f)
        except json.JSONDecodeError as e:
            print(f"[ERROR] JSON decoding failed for {json_path.name}: {e}")
            return pd.DataFrame()

    preds = []
    
    # BDD JSON structure is a list of frame objects
    for frame_data in bdd_frames:
        frame_idx = frame_data.get('frameIndex')
        if frame_idx is None: continue

        for label in frame_data.get('labels', []):
            category = str(label.get('category', 'unknown')).lower()
            
            # Check class mapping
            if category not in BDD_CLASS_MAP or BDD_CLASS_MAP[category] == 'IGNORE':
                continue
            kitti_cls = BDD_CLASS_MAP[category]
            
            box = label.get('box2d')
            score = label.get('score', 1.0) # Assume 1.0 if score is missing
            track_id = label.get('id', -1)  # Track ID is mandatory
            
            if box and track_id != -1:
                x1, y1, x2, y2 = box['x1'], box['y1'], box['x2'], box['y2']
                
                preds.append({
                    "frame": int(frame_idx),
                    "id": int(track_id),
                    "type": kitti_cls,
                    "x": float(x1),
                    "y": float(y1),
                    "w": float(x2 - x1),
                    "h": float(y2 - y1),
                    "conf": float(score)
                })
            
    return pd.DataFrame(preds)


def export_seq_to_kitti_result(pred_df: pd.DataFrame, sequence_id: str, output_dir: Path):
    """
    Exports a DataFrame into the 17-field KITTI 2D tracking results plaintext 
    format required by TrackEval (frame id type x1 y1 x2 y2 ... score).
    """
    out = output_dir / f"{sequence_id}.txt"
    rows = []
    
    for _, r in pred_df.iterrows():
        # Calculate x2 and y2 from x, y, w, h
        x1 = float(r['x']); y1 = float(r['y'])
        x2 = float(r['x'] + r['w']); y2 = float(r['y'] + r['h'])
        
        # KITTI Tracking Format (17 fields, plus score):
        # 0: frame, 1: id, 2: type 
        # 3-5: truncated, occluded, alpha (sentinels)
        # 6-9: x1, y1, x2, y2 (pixel coordinates)
        # 10-16: h, w, l, X, Y, Z, ry (sentinel 3D fields)
        # 17: score (confidence)
        rows.append([
            int(r['frame']), int(r['id']), str(r['type']),
            -1, -1, -10, x1, y1, x2, y2,
            -1, -1, -1, -1000, -1000, -1000, -10,
            float(r.get('conf', 1.0))
        ])
    
    # Sort by frame then ID, as required by the format
    rows.sort(key=lambda R: (R[0], R[1]))
    
    with out.open("w") as f:
        for R in rows:
            # Write with precise formatting for the 2D box coordinates and score
            f.write(
                f"{R[0]} {R[1]} {R[2]} {R[3]} {R[4]} {R[5]} "
                f"{R[6]:.2f} {R[7]:.2f} {R[8]:.2f} {R[9]:.2f} "
                f"{R[10]} {R[11]} {R[12]} {R[13]} {R[14]} {R[15]} {R[16]} {R[17]:.6f}\n"
            )
    return out

def convert_predictions():
    """Main function to find JSONs and convert them all."""
    print("="*80)
    print("CONVERTING BDD100K JSONS (PER SEQUENCE) TO KITTI PLAINTEXT (.TXT)")
    print("="*80)
    
    # Create output directory
    OUTPUT_TRKDATA_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Reading JSONs from: {INPUT_JSON_DIR.resolve()}")
    print(f"Writing KITTI TXTs to: {OUTPUT_TRKDATA_DIR.resolve()}\n")

    json_files = list(INPUT_JSON_DIR.glob("*.json"))
    
    if not json_files:
        print(f"ERROR: No prediction JSON files found in {INPUT_JSON_DIR}")
        print("Please ensure your BDD JSONs are in the configured input directory.")
        return

    total_exports = 0
    
    for json_path in tqdm(json_files, desc="Converting Sequences"):
        # Sequence ID is the filename without extension (e.g., '100000.json' -> '100000')
        sequence_id = json_path.stem
        
        try:
            # 1. Load BDD JSON and convert to flat DataFrame
            df = load_and_convert_sequence(json_path)
            
            if df.empty:
                print(f"[SKIP] Sequence {sequence_id}: Empty or invalid data after filtering.")
                continue
                
            # 2. Export flat DataFrame to KITTI .txt file
            export_seq_to_kitti_result(df, sequence_id, OUTPUT_TRKDATA_DIR)
            total_exports += 1

        except Exception as e:
            print(f"[ERROR] Failed to process {sequence_id}: {e}")
            import traceback
            traceback.print_exc()
            
    print("\n--- Summary ---")
    print(f"Successfully converted {total_exports} sequences to KITTI plaintext format.")
    print("Conversion complete. The resulting files are ready for TrackEval.")

if __name__ == "__main__":
    convert_predictions()

CONVERTING BDD100K JSONS (PER SEQUENCE) TO KITTI PLAINTEXT (.TXT)
Reading JSONs from: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K\labelsMOT\val
Writing KITTI TXTs to: C:\Users\Manish\Documents\SpikeYOLOALL\BDD100K_tracking\trackeval_io\trackers\KITTI_2D_BOX\BDD_JSON_TO_KITTI_CONV\data



Converting Sequences: 100%|████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  6.45it/s]


--- Summary ---
Successfully converted 50 sequences to KITTI plaintext format.
Conversion complete. The resulting files are ready for TrackEval.


In [1]:
# ===========================================================
# End-to-end: Predict -> Export -> TrackEval -> Official metrics
# ===========================================================

# ---- numpy compatibility shim (put at top, BEFORE importing trackeval) ----
import numpy as np

# Add removed numpy aliases back (safe: maps to built-in types)
# Only add the ones you need to avoid masking real problems
if not hasattr(np, "float"):
    np.float = float
if not hasattr(np, "int"):
    np.int = int
if not hasattr(np, "bool"):
    np.bool = bool
# ---------------------------------------------------------------------------

import os
from pathlib import Path
import json
import pandas as pd
import cv2
from tqdm import tqdm
from collections import defaultdict
from textwrap import dedent

# --- CONFIG (edit paths as needed) ---
IMAGE_DIR   = Path(r"C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K\videos\val")
GT_ROOT     = Path(r"C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K\labelsMOT")
# CRITICAL FIX: Kitti2DBox requires the GT files to be in a subdirectory named 'label_02'
LABEL_DIR   = GT_ROOT / "label_02" 
OUTPUT_DIR  = Path(r"C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking")

MODEL_PATH  = Path('BDD1280_ep6.pt')
TRACKER_YAML= "botsort.yaml"  
CONF_THRESH = 0.1
SAVE_VIDEOS = True
TRACKER_NAME= "SpikeYOLO"

# Optional: limit sequences for a quick pass (None = all)
MAX_SEQUENCES = None     # e.g., 3

# --- Create folders ---
(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / "videos").mkdir(exist_ok=True)
(OUTPUT_DIR / "predictions").mkdir(exist_ok=True)
(OUTPUT_DIR / "metrics").mkdir(exist_ok=True)

# TrackEval IO layout
TE_ROOT           = OUTPUT_DIR / "trackeval_io"
TE_TRACKERS_ROOT = TE_ROOT / "trackers" / "KITTI_2D_BOX"
TE_TRKDATA        = TE_TRACKERS_ROOT / TRACKER_NAME / "data"
TE_TRKDATA.mkdir(parents=True, exist_ok=True)

RAW_OUT = OUTPUT_DIR / "metrics" / "trackeval_raw"
RAW_OUT.mkdir(parents=True, exist_ok=True)

# The seqmap must be created in the GT root for Kitti2DBox to find it
SEQMAP_FILE = GT_ROOT / "evaluate_tracking.seqmap.training"

# --- Imports that can be heavy ---
from ultralytics import YOLO
import trackeval
from scipy.optimize import linear_sum_assignment  

# ========== 1) Utilities ==========

CLASS_MAPPING = {
    'Car': 'Car', 'Van': 'Car', 'Truck': 'Car', 'Tram': 'Car',
    'Pedestrian': 'Pedestrian', 'Person_sitting': 'Pedestrian',
    'Cyclist': 'IGNORE', 'Misc': 'IGNORE', 'DontCare': 'IGNORE'
}
VALID_CLASSES = ['Car', 'Pedestrian']

def get_sequences(image_dir: Path):
    # Ensure the GT label directory exists before checking sequences
    if not LABEL_DIR.exists():
        print(f"Warning: GT label directory not found at {LABEL_DIR}. Creating it...")
        LABEL_DIR.mkdir(parents=True, exist_ok=True)
        # If it was just created, it has no labels, which will lead to a failure later.
        
    seqs = sorted([d.name for d in image_dir.iterdir() if d.is_dir()])
    if MAX_SEQUENCES is not None:
        seqs = seqs[:MAX_SEQUENCES]
    print(f"Found {len(seqs)} sequences in {IMAGE_DIR.name}: {seqs[:5]}{' ...' if len(seqs)>5 else ''}")
    return seqs

def setup_model_mapping(model):
    """Map model classes to KITTI classes."""
    model_to_kitti = {}
    target_ids       = []
    print("Detected model classes:", model.names)
    for cls_id, name in model.names.items():
        n = name.lower().strip()
        if n in ['car','van','truck','tram','vehicle']:
            model_to_kitti[cls_id] = 'Car'
            target_ids.append(cls_id)
        elif n in ['pedestrian','person','person_sitting']:
            model_to_kitti[cls_id] = 'Pedestrian'
            target_ids.append(cls_id)
    print("Class mapping (model -> KITTI):", {model.names[i]:model_to_kitti[i] for i in model_to_kitti})
    return model_to_kitti, target_ids

# This function is now only used for verification/debugging and not the main GT loading loop
def load_kitti_labels(sequence_id: str):
    file = LABEL_DIR / f"{sequence_id}.txt"
    if not file.exists():  
        return None
    rows = []
    with open(file, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 10:  
                continue
            frame = int(p[0]); tid = int(p[1]); obj = p[2]
            if obj not in CLASS_MAPPING:  
                continue
            mapped = CLASS_MAPPING[obj]
            if mapped == "IGNORE":  
                continue
            # KITTI fields
            # p[3] = truncated (0-1)
            # p[4] = occluded (0-3)
            # p[5] = alpha (-pi to pi)
            x1,y1,x2,y2 = map(float, p[6:10]) # Bbox (x1, y1, x2, y2)
            rows.append({
                "frame": frame, "id": tid, "type": mapped,
                "x": x1, "y": y1, "w": x2-x1, "h": y2-y1
            })
    df = pd.DataFrame(rows)
    return df

def export_seq_to_kitti_result(pred_df: pd.DataFrame, sequence_id: str):
    """
    KITTI 2D tracking results format (per-seq file):
    frame id type truncated occluded alpha x1 y1 x2 y2 h w l X Y Z ry score
    Unknown 3D fields filled with sentinels.
    """
    out = TE_TRKDATA / f"{sequence_id}.txt"
    rows = []
    for _, r in pred_df.iterrows():
        x1 = float(r['x']); y1 = float(r['y'])
        x2 = float(r['x'] + r['w']); y2 = float(r['y'] + r['h'])
        rows.append([
            int(r['frame']), int(r['id']), str(r['type']),
            -1, -1, -10, x1, y1, x2, y2,
            -1, -1, -1, -1000, -1000, -1000, -10,
            float(r.get('conf', 1.0))
        ])
    rows.sort(key=lambda R: (R[0], R[1]))
    with out.open("w") as f:
        for R in rows:
            f.write(
                f"{R[0]} {R[1]} {R[2]} {R[3]} {R[4]} {R[5]} "
                f"{R[6]:.2f} {R[7]:.2f} {R[8]:.2f} {R[9]:.2f} "
                f"{R[10]} {R[11]} {R[12]} {R[13]} {R[14]} {R[15]} {R[16]} {R[17]:.6f}\n"
            )
    return out

def build_seqmap_for_present_sequences():
    """
    Build seqmap only for sequences that have BOTH GT and tracker files.
    """
    # Use the specific output data paths
    trk_seqs = sorted(p.stem for p in TE_TRKDATA.glob("*.txt"))
    # Use the updated LABEL_DIR path (GT_ROOT / "label_02")
    gt_seqs  = sorted(p.stem for p in LABEL_DIR.glob("*.txt"))
    present  = sorted(set(trk_seqs).intersection(gt_seqs))
    
    if not present:
        # Improved error message to reflect the cause
        print(f"\n--- DEBUG INFO ---")
        print(f"GT Folder: {LABEL_DIR} (Files: {len(gt_seqs)})")
        print(f"Tracker Folder: {TE_TRKDATA} (Files: {len(trk_seqs)})")
        print(f"Common Sequences: {present}")
        print(f"--- END DEBUG INFO ---")
        raise RuntimeError("No overlapping sequences between tracker results and GT. Ensure that for every sequence in 'TE_TRKDATA', a corresponding label file exists in 'LABEL_DIR' ('label_02' folder).")

    rows = []
    for seq in present:
        gt_file = LABEL_DIR / f"{seq}.txt"
        # Read the file to find max frame number
        try:
            df = pd.read_csv(gt_file, sep=r"\s+", header=None, usecols=[0], engine="python")
            if df.empty:  
                continue
            max_f = int(df.iloc[:,0].max())
            rows.append((seq, 0, max_f+1)) # Frame end is inclusive
        except Exception as e:
            print(f"Warning: Could not read GT file {seq}.txt: {e}")
            continue

    if not rows:
        raise RuntimeError("No non-empty GT sequences to evaluate after filtering.")
        
    with SEQMAP_FILE.open("w") as f:
        for seq, s, e in rows:
            # Format: sequence_id empty start_frame end_frame
            f.write(f"{seq} empty {s:06d} {e:06d}\n") 
            
    print(f"✓ Wrote seqmap with {len(rows)} sequences at: {SEQMAP_FILE}")

def _color_for_class(class_name: str, track_id: int):
    # Deterministic color based on ID for visual tracking consistency
    np.random.seed(track_id)
    base = np.array([0, 255, 0]) if class_name == 'Car' else np.array([255, 100, 0])
    return tuple(map(int, np.clip(base + np.random.randint(-30, 30, 3), 0, 255)))

def _ensure_bgr_uint8(img):
    if img is None:
        return None
    if img.dtype != np.uint8:
        img = np.clip(img, 0, 255).astype(np.uint8)
    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    elif img.ndim == 3 and img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return img

# ========== 2) Predict all sequences (YOLO+BOTSort; stream + GMC disabled; skip processed) ==========
print("\n=== Running predictions (YOLO + BOTSort) ===")
sequences = get_sequences(IMAGE_DIR)
model = YOLO(str(MODEL_PATH))
model_to_kitti, target_ids = setup_model_mapping(model)

# --- Prepare a temporary tracker YAML with GMC turned off ---
tmp_tracker_yaml = OUTPUT_DIR / "botsort_no_gmc.yaml"
trk_path = Path(TRACKER_YAML)

BOT_SORT_DEFAULTS = """
proximity_thresh: 0.5
appearance_thresh: 0.25
"""
GMC_OVERRIDE = "gmc_method: none"

if trk_path.exists():
    with open(trk_path, "r") as f:
        orig_yaml = f.read()

    new_yaml = orig_yaml
    for line in BOT_SORT_DEFAULTS.strip().split('\n'):
        key = line.split(':')[0].strip()
        if f"{key}:" not in new_yaml:
            new_yaml += f"\n{line}"

    lines = new_yaml.splitlines()
    found_gmc_method = False
    final_lines = []
    
    for ln in lines:
        if ln.strip().startswith("gmc_method:"):
            final_lines.append(GMC_OVERRIDE)
            found_gmc_method = True
        elif ln.strip().startswith("gmc:"):
            continue
        else:
            final_lines.append(ln)

    new_yaml = "\n".join(final_lines)

    if not found_gmc_method:
        new_yaml += f"\n{GMC_OVERRIDE}\n"

else:
    # No tracker YAML provided -> generate a minimal, sane BoT-SORT config with GMC off
    new_yaml = dedent(f"""
        tracker_type: botsort
        track_high_thresh: 0.1
        track_low_thresh: 0.05
        new_track_thresh: 0.2
        match_thresh: 0.8
        track_buffer: 30
        with_reid: false
        proximity_thresh: 0.5
        appearance_thresh: 0.25
        {GMC_OVERRIDE}
    """).strip()

with open(tmp_tracker_yaml, "w") as f:
    f.write(new_yaml)

print(f"Using tracker config: {tmp_tracker_yaml}")

all_pred_files = []
sequences_with_gt = [] # Only track sequences that have GT
for seq_id in sequences:
    # --- FIX 1: Check for corresponding GT file before starting prediction ---
    gt_file = LABEL_DIR / f"{seq_id}.txt"
    if not gt_file.exists():
        print(f"\n--- Sequence {seq_id} ---")
        print(f"  (GT file missing: {gt_file.name} in {LABEL_DIR}) -> skip sequence.")
        print(f"  ACTION REQUIRED: Ensure your label files are in a folder named 'label_02' within GT_ROOT.")
        continue

    sequences_with_gt.append(seq_id)
    
    print(f"\n--- Sequence {seq_id} ---")
    seq_dir = IMAGE_DIR / seq_id
    imgs = sorted(seq_dir.glob("*.jpg"))
    if not imgs:
        print("  (no images) -> skip")
        continue

    # --- Skip if already processed ---
    pred_csv_path = OUTPUT_DIR / "predictions" / f"{seq_id}_predictions.csv"
    te_file_path  = TE_TRKDATA / f"{seq_id}.txt"
    video_path    = OUTPUT_DIR / "videos" / f"{seq_id}_tracked.mp4"
    already_done  = pred_csv_path.exists() and te_file_path.exists() and (not SAVE_VIDEOS or video_path.exists())
    if already_done:
        print("  ✓ Already processed (found predictions + TrackEval file"
              f"{' + video' if SAVE_VIDEOS else ''}). Skipping.")
        all_pred_files.append(pred_csv_path)
        continue

    # Reset any lingering tracker state (IMPORTANT for sequence-by-sequence tracking)
    try:
        if hasattr(model, "predictor") and hasattr(model.predictor, "reset_tracking"):
            model.predictor.reset_tracking()
    except Exception:
        pass

    # Probe first frame to set video size
    probe = _ensure_bgr_uint8(cv2.imread(str(imgs[0]), cv2.IMREAD_COLOR))
    if probe is None:
        print(f"  (cannot read first frame: {imgs[0]}) -> skip")
        continue
    ref_h, ref_w = probe.shape[:2]

    # Video writer
    vw = None
    if SAVE_VIDEOS:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        # NOTE: Frame rate is hardcoded to 10 FPS
        vw = cv2.VideoWriter(str(video_path), fourcc, 10, (ref_w, ref_h))

    preds = []
    frame_idx = -1

    # Single efficient streaming call over the folder
    for res in tqdm(
        model.track(
            source=str(seq_dir),
            conf=CONF_THRESH,
            iou=0.5,
            tracker=str(tmp_tracker_yaml),      # GMC disabled
            classes=target_ids if target_ids else None,
            persist=False,
            verbose=False,
            stream=True
        ),
        total=len(imgs),
        desc=f"Tracking {seq_id}"
    ):
        frame_idx += 1
        frame = _ensure_bgr_uint8(res.orig_img)
        if frame.shape[:2] != (ref_h, ref_w):
            frame = cv2.resize(frame, (ref_w, ref_h), interpolation=cv2.INTER_LINEAR)

        boxes = res.boxes
        if boxes is not None and boxes.id is not None:
            xyxy = boxes.xyxy.cpu().numpy()
            tids = boxes.id.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            clss = boxes.cls.cpu().numpy()
            for box, tid, conf, cls_idx in zip(xyxy, tids, confs, clss):
                x1, y1, x2, y2 = box
                k = int(cls_idx)
                if k not in model_to_kitti:
                    continue
                kitti_cls = model_to_kitti[k]
                preds.append({
                    "frame": frame_idx,
                    "id": int(tid),
                    "type": kitti_cls,
                    "x": float(x1),
                    "y": float(y1),
                    "w": float(x2 - x1),
                    "h": float(y2 - y1),
                    "conf": float(conf)
                })
                if SAVE_VIDEOS:
                    color = _color_for_class(kitti_cls, int(tid))
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                    label = f"{kitti_cls} ID:{int(tid)} {conf:.2f}"
                    cv2.putText(frame, label, (int(x1), int(y1) - 5),
                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        if SAVE_VIDEOS and vw is not None:
            vw.write(frame)

    if SAVE_VIDEOS and vw is not None:
        vw.release()
        print(f"  ✓ Video saved: {video_path}")

    pred_df = pd.DataFrame(preds)
    if pred_df.empty:
        print("  (no predictions) -> skip export")
        continue

    # Save predictions CSV
    pred_df.to_csv(pred_csv_path, index=False)
    all_pred_files.append(pred_csv_path)
    print(f"  ✓ Predictions CSV: {pred_csv_path} ({len(pred_df)} dets)")

    # Export to TrackEval KITTI result file
    te_file = export_seq_to_kitti_result(pred_df, seq_id)
    print(f"  ✓ TrackEval file: {te_file}")

if not all_pred_files:
    raise RuntimeError("No predictions produced (or no sequences had GT files); check model path / tracker / confidence threshold / GT path.")


# ========== 3) Build seqmap (reconcile GT & tracker files) ==========
build_seqmap_for_present_sequences()

# ========== 4) Run TrackEval (official) ==========
print("\n=== Running TrackEval (official KITTI_2D_BOX) ===")
dataset_config = {
    'GT_FOLDER'         : str(GT_ROOT),         # TrackEval looks in GT_ROOT / 'label_02'
    'TRACKERS_FOLDER' : str(TE_TRACKERS_ROOT),  # TrackEval looks in TRACKERS_FOLDER / TRACKER_NAME / 'data'
    'OUTPUT_FOLDER'   : str(RAW_OUT),
    'TRACKERS_TO_EVAL': [TRACKER_NAME],
    'BENCHMARK'       : 'KITTI_2D_BOX',
    'SEQMAP_FILE'     : str(SEQMAP_FILE),
    'DO_PREPROC'      : False,
    'CLASSES_TO_EVAL' : ['Car', 'Pedestrian'],
}
eval_config = {
    'USE_PARALLEL': False,
    'NUM_PARALLEL_CORES': 1,
    'PRINT_RESULTS': True,
    'PRINT_ONLY_COMBINED': False,
    'TIME_PROGRESS': False,
    'DISPLAY_LESS_PROGRESS': True,
}
metrics_list = [trackeval.metrics.HOTA(), trackeval.metrics.CLEAR(), trackeval.metrics.Identity()]
evaluator     = trackeval.Evaluator(eval_config)
dataset_list = [trackeval.datasets.Kitti2DBox(dataset_config)]
output_res, _ = evaluator.evaluate(dataset_list, metrics_list)

# ========== 5) Parse nested results and compute official AUCs ==========
def arr_mean(x):
    try:
        x = np.asarray(x)
        if x.ndim == 0:
            return float(x)
        return float(np.nanmean(x))
    except Exception:
        return None

CLASS_NAME_MAP = {"car": "Car", "pedestrian": "Pedestrian"}

rows = []
for dataset_name, trackers in output_res.items():
    for tracker_name, seqs in trackers.items():
        for seq_name, metrics in seqs.items():
            for cls_key, cls_block in metrics.items():
                if not isinstance(cls_block, dict):
                    continue
                cls_norm = str(cls_key).lower()
                if cls_norm not in CLASS_NAME_MAP:
                    continue
                pretty_cls = CLASS_NAME_MAP[cls_norm]

                # HOTA group
                HOTA_grp = cls_block.get("HOTA", {})
                HOTA_auc = arr_mean(HOTA_grp.get("HOTA"))
                DetA_auc = arr_mean(HOTA_grp.get("DetA"))
                AssA_auc = arr_mean(HOTA_grp.get("AssA"))
                LocA_auc = arr_mean(HOTA_grp.get("LocA")) if "LocA" in HOTA_grp else None

                # CLEAR group
                CLEAR = cls_block.get("CLEAR", {})
                MOTA  = CLEAR.get("MOTA", None)
                MOTP  = CLEAR.get("MOTP", None)
                MOTA  = float(MOTA) if MOTA is not None else None
                MOTP  = float(MOTP) if MOTP is not None else None

                # Identity group
                ID = cls_block.get("Identity", {})
                IDF1 = float(ID.get("IDF1")) if "IDF1" in ID else None
                IDP  = float(ID.get("IDP"))  if "IDP"  in ID else None
                IDR  = float(ID.get("IDR"))  if "IDR"  in ID else None

                rows.append({
                    "Sequence": seq_name,
                    "Class": pretty_cls,
                    "HOTA": HOTA_auc,
                    "DetA": DetA_auc,
                    "AssA": AssA_auc,
                    "LocA": LocA_auc,
                    "MOTA": MOTA,
                    "MOTP": MOTP,
                    "IDF1": IDF1,
                    "IDP": IDP,
                    "IDR": IDR
                })

metrics_df = pd.DataFrame(rows).sort_values(["Sequence", "Class"])
if metrics_df.empty:
    raise RuntimeError("Parsed metrics are empty. The expected nested keys changed; print output_res to inspect.")

# Save per-seq, per-class metrics
per_seq_csv = OUTPUT_DIR / "metrics" / "trackeval_per_sequence_per_class.csv"
metrics_df.to_csv(per_seq_csv, index=False)
print(f"\n✓ Saved per-seq, per-class metrics: {per_seq_csv}")

# ========== 6) Print per-sequence with BOTH classes side-by-side ==========
def print_seq(seq):
    d = metrics_df[metrics_df["Sequence"]==seq]
    if d.empty:  
        return
    cols = ["Class", "HOTA","DetA","AssA","MOTA","MOTP","IDF1"]
    present = ["Class"] + [c for c in cols if c in d.columns and c != "Class"]
    tab = d[present].set_index("Class")
    # Show decimals (0-1 range) like TrackEval default
    print(f"\nSequence {seq} — OFFICIAL metrics (AUC where applicable)")
    # Note: Using print(tab.to_markdown()) instead of display(tab) for console compatibility
    print(tab.to_markdown(floatfmt=".4f"))

print("\n=== Per-sequence OFFICIAL metrics ===")
for seq in sorted(metrics_df["Sequence"].unique()):
    print_seq(seq)

# ========== 7) Class-wise averages (mean ± std across sequences) ==========
agg = (metrics_df
       .groupby("Class")
       .agg({c:["mean","std"] for c in ["HOTA","DetA","AssA","MOTA","MOTP","IDF1","IDP","IDR"] if c in metrics_df.columns})
       .sort_index())
agg.columns = [f"{m}_{stat}" for m,stat in agg.columns]
class_avg_csv = OUTPUT_DIR / "metrics" / "trackeval_class_averages.csv"
agg.to_csv(class_avg_csv)
print("\n=== Class-wise averages (OFFICIAL) ===")
# Note: Using print(agg.to_markdown()) instead of display(agg) for console compatibility
print(agg.to_markdown(floatfmt=".4f"))
print(f"✓ Saved class averages: {class_avg_csv}")

# Also save compact JSON
class_avg_json = OUTPUT_DIR / "metrics" / "trackeval_class_averages.json"
class_json = {}
for cls, row in agg.iterrows():
    stats = {}
    for m in ["HOTA","DetA","AssA","MOTA","MOTP","IDF1","IDP","IDR"]:
        mean_col = f"{m}_mean"; std_col = f"{m}_std"
        if mean_col in agg.columns:
            stats[m] = {"mean": float(row[mean_col]), "std": float(row.get(std_col, np.nan))}
    class_json[cls] = stats
with open(class_avg_json, "w") as f:
    json.dump(class_json, f, indent=2)
print(f"✓ Saved class averages JSON: {class_avg_json}")

# ========== 8) Dataset-wide combined (macro) ==========
# Average across all (seq, class) rows
combined = metrics_df.drop(columns=["Sequence","Class"]).mean(numeric_only=True)
print("\n=== Combined (macro average over sequences & classes) ===")
print(combined.to_string(float_format=lambda x: f"{x:0.4f}"))

combined_csv = OUTPUT_DIR / "metrics" / "trackeval_combined_macro.csv"
combined.to_csv(combined_csv)
print(f"✓ Saved combined macro averages: {combined_csv}")

C:\Users\Manish\miniconda3\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\Manish\miniconda3\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)



=== Running predictions (YOLO + BOTSort) ===
Found 50 sequences in val: ['b1c81faa-3df17267', 'b1cd1e94-549d0bfe', 'b1ceb32e-3f481b43', 'b1ceb32e-51852abe', 'b1d0091f-f2c2d2ae'] ...
Detected model classes: {0: 'car', 1: 'pedestrian'}
Class mapping (model -> KITTI): {'car': 'Car', 'pedestrian': 'Pedestrian'}
Using tracker config: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\botsort_no_gmc.yaml

--- Sequence b1c81faa-3df17267 ---


Tracking b1c81faa-3df17267:   0%|                                                              | 0/202 [00:00<?, ?it/s]C:\Users\Manish\miniconda3\Lib\site-packages\torch\functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4319.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Tracking b1c81faa-3df17267: 100%|████████████████████████████████████████████████████| 202/202 [00:54<00:00,  3.69it/s]


  ✓ Video saved: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\videos\b1c81faa-3df17267_tracked.mp4
  ✓ Predictions CSV: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\predictions\b1c81faa-3df17267_predictions.csv (391 dets)
  ✓ TrackEval file: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\trackeval_io\trackers\KITTI_2D_BOX\SpikeYOLO\data\b1c81faa-3df17267.txt

--- Sequence b1cd1e94-549d0bfe ---


Tracking b1cd1e94-549d0bfe: 100%|████████████████████████████████████████████████████| 203/203 [00:55<00:00,  3.63it/s]


...

Tracking b251064f-30002542: 100%|████████████████████████████████████████████████████| 202/202 [00:39<00:00,  5.16it/s]


  ✓ Video saved: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\videos\b251064f-30002542_tracked.mp4
  ✓ Predictions CSV: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\predictions\b251064f-30002542_predictions.csv (1531 dets)
  ✓ TrackEval file: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\trackeval_io\trackers\KITTI_2D_BOX\SpikeYOLO\data\b251064f-30002542.txt

--- Sequence b255cd6c-0bdf0ac7 ---


Tracking b255cd6c-0bdf0ac7: 100%|████████████████████████████████████████████████████| 202/202 [00:52<00:00,  3.87it/s]


  ✓ Video saved: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\videos\b255cd6c-0bdf0ac7_tracked.mp4
  ✓ Predictions CSV: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\predictions\b255cd6c-0bdf0ac7_predictions.csv (598 dets)
  ✓ TrackEval file: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K_tracking\trackeval_io\trackers\KITTI_2D_BOX\SpikeYOLO\data\b255cd6c-0bdf0ac7.txt
✓ Wrote seqmap with 50 sequences at: C:\Users\Manish\Documents\SpikeYOLOPaper\BDD100K\labelsMOT\evaluate_tracking.seqmap.training

=== Running TrackEval (official KITTI_2D_BOX) ===

CLEAR Config:
THRESHOLD            : 0.5                           
PRINT_CONFIG         : True                          

Identity Config:
THRESHOLD            : 0.5                           
PRINT_CONFIG         : True                          

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
PRINT_RESULTS        : True                       

<Figure size 640x480 with 0 Axes>